In [1]:
!pip install schedule

In [2]:
!pip install tensorflow

In [3]:
!pip install konlpy

In [4]:
!pip install h5py --upgrade

In [5]:
from selenium import webdriver
import schedule
import time
from datetime import datetime, timedelta
import sqlite3
from bs4 import BeautifulSoup
import tensorflow as tf
import konlpy
import pickle
from konlpy.tag import Okt


def job():  
    driver = webdriver.Chrome('./chromedriver.exe')
    conn = sqlite3.connect('./db.stock')
    c = conn.cursor()
    c.execute("CREATE TABLE IF NOT EXISTS article (id INTEGER PRIMARY KEY AUTOINCREMENT, date TEXT, time TEXT, title TEXT, content TEXT, press TEXT , href TEXT, stock TEXT, posi_nega TEXT)")

    uri = 'https://finance.naver.com/news/market_special.nhn?&page='
    fnews = []

    for page in range(1, 4):
        target = uri+str(page)
        driver.get(target)

        try:
            for n in range(1, 24):
    #             time.sleep(2)
                driver.implicitly_wait(5)
                try: 
                    driver.find_element_by_xpath('//*[@id="contentarea_left"]/div[2]/table/tbody/tr[%s]/td[1]/a' %n).click()
                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')

                    presses = soup.select('span.press > img')
                    press = presses[0]['title']

                    datas = soup.select('div.article_info > h3')
                    title2 = datas[0].text.strip()
                    title1 = title2.replace("'","")
                    title = title1.replace('"','')

                    dates = soup.select('div.article_sponsor > span')
                    date = dates[0].text.strip()


                    contents = soup.select('div#content')
                    content1 = contents[0]
                    for tag in content1.find_all(['li', 'h3']):
                        tag.replace_with('')
                        content = content1.text.strip()

                    driver.back()

                    html = driver.page_source
                    soup1 = BeautifulSoup(html, 'html.parser')

                    times = soup1.select('td.wdate')
                    time1 = times[int(n)-1]
                    time = time1.text.strip()[-5:]

                    hrefs = soup1.select('td.publicSubject > a')
                    href1 = hrefs[int(n)-1]
                    href = 'https://finance.naver.com'+ href1['href']
                    

                    date_int = int(date.replace('-',''))
    
                    yesterday1 = datetime.today() - timedelta(1)
                    yesterday = int(yesterday1.strftime("%Y%m%d"))
            
        
                    loaded_model = tf.keras.models.load_model('./naver_finance_news_deeplearnig.h5')
                    stopwords = pickle.load(open('./stopwords.pkl','rb'))
                    tokenizer = pickle.load(open('./tokenizer.pkl','rb'))
                    
                    okt = konlpy.tag.Okt()
                    new_sentence = okt.morphs(title, stem=True)
                    new_sentence = [ tok for tok in new_sentence if tok not in stopwords ]
                    encoded = tokenizer.texts_to_sequences([new_sentence])
                    pad_new = tf.keras.preprocessing.sequence.pad_sequences(encoded, maxlen=50)
                    score = loaded_model.predict(pad_new)
                    if(score > 0.5):
                      posi_nega = 1
                    else:
                      posi_nega = 0
                    
                    a = [date , time , press , title, content, href, posi_nega]
                    fnews.append(a)
                
                    if date_int == yesterday:
                        c.execute("INSERT INTO article( date , time , title, content, press, href, stock , posi_nega ) VALUES(?,?,?,?,?,?,?,?)",(date, time, title, content, press, href,'stock',posi_nega))
                    else:
                        continue

                except:
                    pass

        except:
            pass  


    conn.commit()
    c.close()
    print(fnews)
    
schedule.every().day.at("06:00").do(job) 

while True:
    schedule.run_pending()
    time.sleep(5)


[['2021-08-19', '15:55', '연합뉴스', '크래프톤 5% 급등…장중 공모가 회복(종합)', "크래프톤, 코스피 상장역대 2위의 공모 규모에도 청약 흥행에서 참패한 게임업체 크래프톤이 유가증권시장에 상장한 지난 10일 오후 서울 여의도 한국거래소 외벽에 크래프톤 상장 축하 현수막이 걸려 있는 모습.[연합뉴스 자료사진](서울=연합뉴스) 박진형 기자 = 상장 이후 주가가 공모가 밑으로 떨어졌던 게임업체 크래프톤이 19일 급등, 장중 한때 공모가를 회복했다.    이날 유가증권시장에서 크래프톤은 전날보다 5.01% 상승한 49만2천500에 거래를 마쳤다.    지난 10일 상장한 크래프톤은 이날 장중 한때 49만8천500원까지 뛰어올라 7거래일만에 공모가(49만8천원)를 넘어서기도 했다.    앞서 지난 17일 크래프톤은 인도 시장에 내놓은 모바일 게임 '배틀그라운드 모바일 인도'가 출시 44일 만에 누적 다운로드 5천만회를 돌파했다고 발표했다    이 게임은 지난달 2일 구글플레이 출시 이후 일주일 만에 누적 이용자수 3천400만명, 일일 최대 이용자수 1천600만명, 최대 동시 접속자수 240만명을 각각 기록했다.     크래프톤은 또 2분기 연결기준 영업이익이 1천742억원으로 엔씨소프트, 넷마블, 넥슨 등 이른바 '3N' 게임사들을 추월했다.    jhpark@yna.co.kr", 'https://finance.naver.com/news/market_special_read.nhn?article_id=0012608954&office_id=001&type=title&keyword=', 1], ['2021-08-19', '15:50', '연합뉴스', '롯데렌탈, 상장일 공모가 하회…브레인즈컴퍼니 따상(종합)', "롯데렌탈 공모주 청약NH투자증권 영업점에 롯데렌탈 공모주 청약을 알리는 안내판이 서 있다. [NH투자증권 제공. 재판매 및 DB금지](서울=연합뉴스) 김태종 기자 = 19일 유가증권시장에 상장한 롯데렌탈이 공모가 아래에서 첫 거래를 마감했

KeyboardInterrupt: 

In [6]:
import pandas as pd

In [8]:
conn = sqlite3.connect('./db.stock')
c = conn.cursor()
data = pd.read_sql('SELECT * FROM article', con=conn)
c.close()

data

,id,date,time,title,content,press,href,stock,posi_nega
0,1,2021-08-19,15:55,크래프톤 5% 급등…장중 공모가 회복(종합),"크래프톤, 코스피 상장역대 2위의 공모 규모에도 청약 흥행에서 참패한 게임업체 크래...",연합뉴스,https://finance.naver.com/news/market_special_...,stock,posi_nega
1,2,2021-08-19,15:50,"롯데렌탈, 상장일 공모가 하회…브레인즈컴퍼니 '따상'(종합)",롯데렌탈 공모주 청약NH투자증권 영업점에 롯데렌탈 공모주 청약을 알리는 안내판이 서...,연합뉴스,https://finance.naver.com/news/market_special_...,stock,posi_nega
2,3,2021-08-19,14:07,"IBKS제14호스팩, 에스에이티 흡수합병 소식에 ‘급등’",[이데일리 유준하 기자] IBKS제14호스팩(351320)이 19일 디스플레이 장비...,이데일리,https://finance.naver.com/news/market_special_...,stock,posi_nega
3,4,2021-08-19,12:00,크래프톤 6% 급등…공모가 회복,"크래프톤, 코스피 상장역대 2위의 공모 규모에도 청약 흥행에서 참패한 게임업체 크래...",연합뉴스,https://finance.naver.com/news/market_special_...,stock,posi_nega
4,5,2021-08-19,10:52,"유니켐, 강세…2분기 호실적에 골프장 건설 순항",[이데일리 박정수 기자] 유니켐(011330)이 강세를 보인다. 2분기 호실적과 함...,이데일리,https://finance.naver.com/news/market_special_...,stock,posi_nega
...,...,...,...,...,...,...,...,...,...
122,123,2021-08-19,09:20,"브레인즈컴퍼니, 코스닥 상장 첫날 급등..한때 따상도 성공",[파이낸셜뉴스] 지능형 정보기술(IT)인프라 통합관리 솔루션 브레인즈컴퍼니가 코스닥...,파이낸셜뉴스,https://finance.naver.com/news/market_special_...,stock,1
123,124,2021-08-19,09:20,"플래티어, 15%대 강세…4만원대 안착",[이데일리 김인경 기자] 플래티어(367000)가 19일 장 초반 15%대 강세를 ...,이데일리,https://finance.naver.com/news/market_special_...,stock,1
124,125,2021-08-19,09:14,"천보, 공장 증설 기대에 4%대 강세…24만원대 되찾아",[이데일리 김인경 기자] 천보(278280)가 19일 장 초반 강세를 보이고 있다....,이데일리,https://finance.naver.com/news/market_special_...,stock,1
125,126,2021-08-19,09:12,"브레인즈컴퍼니, 코스닥 상장 첫 날 급등",[이데일리 권효중 기자] 지능형 IT 인프라 솔루션을 제공하는 브레인즈컴퍼니(099...,이데일리,https://finance.naver.com/news/market_special_...,stock,1


In [9]:
data.to_csv('./saves/naver_news_data_PosiNega.csv', index=False, encoding='utf-8-sig')